# Ниязов Алдияр М.19-04 
Задание 1 Предсказание количества комментариев к посту в FaceBook

Произведем выгрузку данных из датасета

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('Dataset/Training/Features_Variant_1.csv')

Посмотрим на них

In [2]:
data

,634995,0,463,1,0.0,806.0,11.291044776119403,1.0,70.49513846124168,0.0.1,...,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,1.2,0.19
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,0,0,0,0,0,1,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,1,0,0,0,0,0,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,1,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40943,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,0,1,0,1
40944,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,0,1,0,2
40945,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,1,0,0,72
40946,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,0,1,0,28


Так как данные в датасете отсортированы и это негативно скажется на обучении модели, то перемешаем их

In [3]:
data = data.sample(frac=1)
data.head()

,634995,0,463,1,0.0,806.0,11.291044776119403,1.0,70.49513846124168,0.0.1,...,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,1.2,0.19
12578,1616,0,18,24,0.0,5.0,1.222222,1.0,1.547599,0.0,...,0,0,0,0,0,0,1,0,0,0
30154,1959278,0,12641,18,0.0,513.0,22.871111,13.0,46.630141,0.0,...,0,0,0,0,1,0,0,0,0,0
19802,1457476,0,3961,10,0.0,55.0,4.390805,2.0,9.149281,0.0,...,0,0,0,0,0,1,0,0,0,0
6598,1422427,0,35610,9,0.0,206.0,33.421622,20.0,38.486299,0.0,...,0,0,0,0,0,0,1,0,0,0
23760,454393,0,66403,96,0.0,213.0,13.921348,3.0,30.365276,0.0,...,0,0,0,0,0,0,0,0,1,0


Отделим значения целевого признака в отдельный массив y

In [4]:
y = data['0.19'].values

Оставшиеся признаки сгруппируем в матрицу Х

In [5]:
data = data.values

In [6]:
X = np.delete(data, 53, axis=1)

Вычислим средние значения признаков (means) и стандартное отклонение (stds) по каждому столбцу матрицы Х. Сгруппируем значения в соответствующие массивы.

In [7]:
means, stds = [np.mean(X, axis=0), np.std(X, axis=0)]

Если у какого-нибудь признака стандартное отклонение равно 0, значит этот признак является константой и не имеет никакого влияния на целевой признак. Определим такой признак в Х и удалим его из выборки:

In [8]:
np.where(stds==0)

(array([37]),)

In [9]:
X = np.delete(X, 37, axis=1)

Пересчитаем means и stds

In [10]:
means, stds = [np.mean(X, axis=0), np.std(X, axis=0)]

Произведем нормировку признаков. От каждого значения матрицы Х отнимем среднее значение по столбцу, а результат поделим на стандартное отклонение этого столбца.

In [11]:
def normalize_features(X):
    for i in range(0, X.shape[0]):
        for j in range(0, X.shape[1]):
            X[i][j] = (X[i][j] - means[j])/stds[j]
    return X

In [12]:
X = normalize_features(X)

Добавим к матрице X столбец из единиц, чтобы не обрабатывать отдельно коэффициент $w_0$ линейной регрессии

In [13]:
X = np.hstack((X, np.ones((len(X), 1))))


Опишем фунциb вычисления метрик качества. Данные показатели характеризует качество применяемой модели

In [14]:
def mserror(y, y_pred):
    return 1/len(y)*sum((y - y_pred) ** 2)

In [15]:
def rmserror(y, y_pred):
    return np.sqrt(mserror(y, y_pred))

In [16]:
def determination(X, y, w):
    return 1 - (sum((y - np.dot(X, w))**2) / sum((y - np.mean(y, axis=0))**2))

Опишем функцию прогноза для линейной модели, принимающую матрицу Х и вектор весов признаков w, возвращающую вектор прогнозов

In [17]:
def linear_prediction(X, w):
    return np.dot(X, w)


Градиент функционала ошибки вычисляется по формуле:

In [18]:
def gradient_step(X, y, w):
    return  np.dot(2/len(y) * X.T, np.add(linear_prediction(X, w), - y))
    

Градиентный спуск будет производиться в цикле по формуле:
    $$\Large w^t = w^{t-1} - lr*gradient step $$ 
    lr - коэффициент шага (learning rate)

In [81]:
def gradient_descent(X, y, w_init, lr, epoch_num):
    weight_dist = np.inf
    w = w_init
    iter_num = 0
    # Склеим матрицу признаков со столбцом из значений целевого признака
    data = np.concatenate((X, y.reshape(len(y), 1)), axis=1)
    # Основной цикл
    for i in range(epoch_num):
        # Перемешаем данные
        np.random.shuffle(data)
        # Разделим таблицу признаков и целевой признак
        X = np.delete(data, 53, axis=1)
        y = np.delete(data, range(53), axis=1).reshape(1, len(y))[0]
        # Пройдемся по датасету batch-ами
        batch_size = 1000
        for batch_idx in range(int(len(y) / batch_size) + 1):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size
            batch = X[start_idx:end_idx]
            y_batch = y[start_idx:end_idx]
            # вычисляем градиент
            grad = gradient_step(batch, y_batch, w)
            # обновляем параметры
            w = w - lr * grad
        
    return w

Для кросс-валидации разделим датасет на 5 частей (примерно по 8190 строк в каждой)

In [88]:
VALIDATION_RATE = 5

In [89]:
data_split = np.array_split(X, VALIDATION_RATE)

In [90]:
target_split = np.array_split(y, VALIDATION_RATE)

In [129]:
def cross_validation(features, target, validation_rate):
    metrics = []
    weights = []
    for idx in range(VALIDATION_RATE):
        data_train = []
        target_train = []
        for i in range(VALIDATION_RATE):
            if i != idx:
                data_train.append(data_split[i])
                target_train.append(target_split[i])
            else:
                data_test = data_split[i]
                target_test = target_split[i]
        data_train = np.concatenate(data_train, axis=0)
        target_train = np.concatenate(target_train)
        # тренировка модели
        grad_desc_weights = gradient_descent(data_train, target_train, np.zeros(X.shape[1]), lr=0.01, epoch_num=10)
        # расчет метрик
        RMSE_train = rmserror(target_train, np.dot(data_train, grad_desc_weights))
        RMSE_test = rmserror(target_test, np.dot(data_test, grad_desc_weights))
        R2_train = determination(data_train, target_train, grad_desc_weights)
        R2_test = determination(data_test, target_test, grad_desc_weights)
        # объединение результатов
        metrics.append([RMSE_train, RMSE_test, R2_train, R2_test])
        weights.append(grad_desc_weights)
    
    return metrics, weights

In [130]:
metrics, weights = cross_validation(data_split, target_split, VALIDATION_RATE)

Расчитаем средние значения и дисперсию по метрикам:

In [131]:
metrics_mean = []
metrics_std = []

for i in range(4):
    result_list = []
    for metric in metrics:
        result_list.append(metric[i])
    metrics_mean.append(np.mean(result_list))
    metrics_std.append(np.std(result_list))


Объединим результаты по метрикам в таблицу:

In [134]:
data_metrics = {
        'fold 1': metrics[0], 
        'fold 2': metrics[1], 
        'fold 3': metrics[2], 
        'fold 4': metrics[3], 
        'fold 5': metrics[4],
        'mean': metrics_mean,
        'std': metrics_std
       }

df = pd.DataFrame(data_metrics, index=['RMSE train', 'RMSE test', 'R2 train', 'R2 test'])
df.to_csv('metrics_result.csv')
df

,fold 1,fold 2,fold 3,fold 4,fold 5,mean,std
RMSE train,29.461518,29.530823,28.398299,30.112106,28.967927,29.294135,0.576550
RMSE test,28.733827,29.344054,32.764651,26.055461,31.431769,29.665953,2.312063
R2 train,0.312818,0.323528,0.332757,0.312323,0.312089,0.318703,0.008245
R2 test,0.337846,0.246463,0.267068,0.337638,0.304139,0.298631,0.036898


In [135]:
data_weights = {
    'fold 1': weights[0], 
    'fold 2': weights[1], 
    'fold 3': weights[2], 
    'fold 4': weights[3], 
    'fold 5': weights[4]
}
indexes = []
for i in range(len(weights[0])):
    indexes.append(f'w{i}')
    
df_weights = pd.DataFrame(data_weights, index=indexes)
df_weights.to_csv('weights_result.csv')

df_weights


,fold 1,fold 2,fold 3,fold 4,fold 5
w0,-0.051660,0.235564,0.291106,0.314745,-1.582701
w1,-0.650947,-0.622038,-0.530142,-0.544059,-0.680442
w2,-1.330902,-1.418288,-1.433825,-1.581432,-0.467589
w3,-0.007882,-0.068634,-0.092870,0.010500,-0.099176
w4,-0.068836,0.557390,-0.498206,-0.229173,-0.201256
w5,0.460543,0.492210,0.283670,0.563678,0.416771
w6,0.368344,0.225562,0.533538,0.467343,0.375193
w7,0.101462,0.204365,0.330617,0.059082,0.119579
w8,1.235328,0.919734,0.960073,1.019815,1.217635
w9,-0.673879,-0.660456,-1.173910,0.265282,-0.604876


Полученные данные по метрикам качества модели объединим в таблицу Results_mini_batch.xls.
Проанализировав полученные результаты, можно сделать вывод: большая часть результатов соответствуют установленным нормам, что говорит об успешном решении поставленной задачи. 